In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm_notebook

import fannypack
from lib import panda_datasets, panda_baseline_models, panda_baseline_training


In [ ]:
# Experiment configuration
experiment_name = "lstm_test"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': True,
    'vision_interval': 2,
}

In [ ]:
print("Creating model...")
model = panda_baseline_models.PandaLSTMModel(batch_size=16)
buddy = fannypack.utils.Buddy(experiment_name, model)

In [ ]:
print("Creating dataset...")
dataset = panda_datasets.PandaParticleFilterDataset(
    'data/pushset_medium.hdf5',
    subsequence_length=16,
    **dataset_args)

In [ ]:
buddy.set_learning_rate(1e-4)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
for _ in tqdm_notebook(range(1000)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader)):
        _, states, observations, _ = fannypack.utils.to_device(batch, buddy._device)

        model.reset_hidden_states(states[:, 0, :])
        predicted_states = model(observations)
        loss = F.mse_loss(predicted_states, states)
        buddy.minimize(loss, checkpoint_interval=500)
        buddy.log("loss", loss)

buddy.save_checkpoint()

In [ ]:
eval_trajectories = panda_datasets.load_trajectories(("data/pushset_small.hdf5",10), **dataset_args)

pred, actual = panda_baseline_training.rollout_lstm(model, eval_trajectories)
panda_baseline_training.eval_rollout(pred, actual, plot=True)

In [ ]:
buddy.save_checkpoint("stable")

In [ ]:
buddy.save_checkpoint()